Now we will be using a Python based web framework called web2py for building APIs and VueJS for building reactive HTML front-end

Web2py inherently supports MVC (Model View Controller) architecture.

Models: define database schema and validations

Views: hosts your HTML code

Controllers: Python code where you will write business logic and manages interactions between front-end and the database.

Even though MVC is a good architectural pattern, the world has moved on to using micro-services architecture so that our front-end is independent of the controller logic (mostly)

Right now, we will be focusing only on the controllers where we will build our APIs

Reference :

    Build Backend API
    https://medium.com/code-heroku/how-to-turn-your-machine-learning-scripts-into-projects-you-can-demo-cbc5611ca442
    
    FrontEnd UI using Vue 
    https://medium.com/code-heroku/part-2-how-to-turn-your-machine-learning-scripts-into-projects-you-can-demo-467b3acff041

# Step 1 : Download web2py folder and item_similarity_df.csv files #

# Step 2 : Go to web2py folder from conda promt and type 'python web2py.py' and hit enter #

Location for me :
C:\Users\Admin\Documents\SelfDrivingCarND\MovieRecommemdation\dataset\CreatingWebUI\web2py

# Step 3 : Select server Ip as local and select any pwd #

# Step 4 : Login to admin at browser and create project #

Location : http://127.0.0.1:8000/admin/default/design/movies#

# Step 5 : Open webpy folder inside atom/visualcode/pycharm and keep your csv file here #
Location (local):C:\Users\Admin\Documents\SelfDrivingCarND\MovieRecommemdation\dataset\CreatingWebUI\web2py\applications\movies\static

# Step 6 : Write your code inside controller section wrt below location #
Location : C:\Users\Admin\Documents\SelfDrivingCarND\MovieRecommemdation\dataset\CreatingWebUI\web2py\applications\movies\controllers

File :default.py

In [5]:
# Step 7: Code in default.py


import pandas as pd

from gluon.tools import Service
service = Service()

def call():   
    return service()

  
def load_recommendations(): 
    item_similarity_df = pd.read_csv("applications/movies/static/item_similarity_df.csv",index_col=0)
    print("item_similarity_df cached in memory")
    return item_similarity_df 

item_similarity_df = cache.ram('item_similarity_df3',load_recommendations,None) 
print(item_similarity_df.head())    

@service.json
##helper method that doesn't recommend a movie if the user has already seen it
def check_seen(recommended_movie,watched_movies):
    for movie_id,movie in watched_movies.items():
        if recommended_movie == movie["title"]:
            return True
    return False

def get_similar_movies(movie_name,user_rating):
    try:
        similar_score = item_similarity_df[movie_name]*(user_rating-2.5)
        similar_movies = similar_score.sort_values(ascending=False)
    except:
        print("don't have movie in model")
        similar_movies = pd.Series([])
    
    return similar_movies


@service.json
def get_recommendations(watched_movies):

    print(watched_movies)
    similar_movies = pd.DataFrame()

    for movie_id,movie in watched_movies.items():
        similar_movies = similar_movies.append(get_similar_movies(movie["title"],movie["rating"]),ignore_index=True)

    all_recommend = similar_movies.sum().sort_values(ascending=False)

    recommended_movies = []
    for movie,score in all_recommend.iteritems():
        if not check_seen(movie,watched_movies):
            recommended_movies.append(movie)    

    if len(recommended_movies) > 100:
        recommended_movies = recommended_movies[0:100]        

    return recommended_movies

Since we do not want to load the data frame every time someone uses our app we will cache it using web2py’s cache.ram method.

# Test your api here #

http://127.0.0.1:8000/movies/static/test.html

Now since , we can see the data , next step is to design the frontend using Vue and html

Download the posters (opensource ) from here :
    https://github.com/babu-thomas/movielens-posters

Check your output :
    http://127.0.0.1:8000/movies/static/movies/index.htmlhttp://127.0.0.1:8000/movies/static/movies/index.html